In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
import os
os.environ['PYSPARK_PYTHON'] = '/Applications/anaconda/bin/python'

In [3]:
import findspark
findspark.init()
print findspark.find()

/usr/local/opt/apache-spark/libexec


In [4]:
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)

In [5]:
import sys
rdd = sc.parallelize(xrange(10),10)
rdd.map(lambda x: sys.version).collect()

['2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.4.0 (x8

In [6]:
sys.version

'2.7.10 |Anaconda 2.4.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]'

In [7]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)

### Read data

In [14]:
frames = []
for i in range(1,20) : 
    dfs = pd.read_csv("2015data/docsdf-"+str(i)+".csv")
    frames.append(dfs)
df = pd.concat(frames)

In [22]:
df.head()

,Unnamed: 0,abstract,date,doctype,headline,id,news_desk,paragraph,section,subsection,text,type
0,0,NaN,2015-10-20,article,"To Reduce the Cost of Drugs, Look to Europe",5624cb2279881072a18d3b60,Upshot,"A system called reference pricing, used in sev...",The Upshot,NaN,"To Reduce the Cost of Drugs, Look to Europe A ...",News
1,1,Editorial reviews recent report by World Healt...,2015-10-28,article,Meat as a Cause of Cancer,56307762798810310043c222,Editorial,A recent report provides persuasive evidence t...,Opinion,NaN,Meat as a Cause of Cancer A recent report prov...,Editorial
2,2,NaN,2015-10-26,multimedia,Meat and Cancer: The W.H.O. Report and What Yo...,562e978e79881079d8f744f4,Health,Answers to a few questions about a report that...,Health,NaN,Meat and Cancer: The W.H.O. Report and What Yo...,Interactive Feature
3,3,World Health Organization scientists recommend...,2015-10-24,article,W.H.O. Panel Backs Polio Vaccine With Just 2 S...,562acef679881010ffa5f64d,Foreign,The scientists also recommended that infants a...,World,NaN,W.H.O. Panel Backs Polio Vaccine With Just 2 S...,Brief
4,4,NaN,2015-10-24,article,Egg Donor Payments,562ac28379881010ffa5f637,Letters,A bioethicist writes that eliminating all guid...,Opinion,NaN,Egg Donor Payments A bioethicist writes that e...,Letter


In [46]:
df.shape

(17936, 12)

In [63]:
sum(df['date'] == 'NaN')

0

In [66]:
sum(df['doctype'] == 'topic')

0

In [78]:
mask = np.ones(len(df), dtype='int')
mask[df['date'] == 'NaN'] = 0
mask = (mask == 1)

/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  from ipykernel import kernelapp as app


In [79]:
mask

array([False,  True,  True, ...,  True,  True,  True], dtype=bool)

In [80]:
df_date = df[mask]

In [77]:
df_date.shape

(17935, 12)

### Clean data

In [17]:
df.dtypes

Unnamed: 0     int64
abstract      object
date          object
doctype       object
headline      object
id            object
news_desk     object
paragraph     object
section       object
subsection    object
text          object
type          object
dtype: object

In [23]:
df['date'] = pd.to_datetime(df['date'])

In [24]:
df.dtypes

Unnamed: 0             int64
abstract              object
date          datetime64[ns]
doctype               object
headline              object
id                    object
news_desk             object
paragraph             object
section               object
subsection            object
text                  object
type                  object
dtype: object

# get rid of common words??

### LDA

In [25]:
from pattern.en import parse
from pattern.en import pprint
from pattern.vector import stem, PORTER, LEMMA
punctuation = list('.,;:!?()[]{}`''\"@#$^&*+-|=~_')

In [26]:
from sklearn.feature_extraction import text 
stopwords=text.ENGLISH_STOP_WORDS

In [27]:
import re
regex1=re.compile(r"\.{2,}")
regex2=re.compile(r"\-{2,}")

In [28]:
def get_parts(thetext):
    thetext=re.sub(regex1, ' ', thetext)
    thetext=re.sub(regex2, ' ', thetext)
    nouns=[]
    descriptives=[]
    for i,sentence in enumerate(parse(thetext, tokenize=True, lemmata=True).split()):
        nouns.append([])
        descriptives.append([])
        for token in sentence:
            #print token
            if len(token[4]) >0:
                if token[1] in ['JJ', 'JJR', 'JJS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    descriptives[i].append(token[4])
                elif token[1] in ['NN', 'NNS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    nouns[i].append(token[4])
    out=zip(nouns, descriptives)
    nouns2=[]
    descriptives2=[]
    for n,d in out:
        if len(n)!=0 and len(d)!=0:
            nouns2.append(n)
            descriptives2.append(d)
    return nouns2, descriptives2

In [30]:
%%time

parseout = []
for index, row in df.iterrows() : 
    parseout.append(get_parts(row.text))

CPU times: user 1min 45s, sys: 399 ms, total: 1min 45s
Wall time: 1min 45s


In [32]:
#nouns are the first elements of the tuples
[e[0] for e in parseout[:3]]

[[],
 [[u'report', u'evidence', u'eating', u'meat', u'cancer', u'risk', u'person'],
  [u'reviews', u'report', u'meat', u'cancer', u'risk', u'person']],
 [[u'question',
   u'report',
   u'colorectal',
   u'cancer',
   u'consumption',
   u'meat',
   u'meat']]]

In [33]:
ldadatardd=sc.parallelize([ele[0] for ele in parseout]).flatMap(lambda l: l)
ldadatardd.cache()
ldadatardd.take(5)

[[u'report', u'evidence', u'eating', u'meat', u'cancer', u'risk', u'person'],
 [u'reviews', u'report', u'meat', u'cancer', u'risk', u'person'],
 [u'question',
  u'report',
  u'colorectal',
  u'cancer',
  u'consumption',
  u'meat',
  u'meat'],
 [u'polio',
  u'vaccine',
  u'scientist',
  u'infant',
  u'month',
  u'measles',
  u'vaccine',
  u'risk'],
 [u'scientist',
  u'polio',
  u'vaccine',
  u'release',
  u'strain',
  u'type',
  u'virus',
  u'paralysis']]

In [34]:
ldadatardd.flatMap(lambda word: word).take(5)

[u'report', u'evidence', u'eating', u'meat', u'cancer']

In [35]:
vocabtups = (ldadatardd.flatMap(lambda word: word)
             .map(lambda word: (word, 1))
             .reduceByKey(lambda a, b: a + b)
             .map(lambda (x,y): x)
             .zipWithIndex()
).cache()

In [36]:
vocab=vocabtups.collectAsMap()
id2word=vocabtups.map(lambda (x,y): (y,x)).collectAsMap()

In [37]:
len(vocab.keys())

10537

In [38]:
#your code here
from collections import defaultdict
def get_count(s) : 
    d = defaultdict(int)
    for w in s : 
        if vocab.has_key(w) :
            i = vocab[w]
            d[i] += 1
    return d.items()
documents = ldadatardd.map(get_count)

In [39]:
documents.take(5)

[[(3201, 1), (6255, 1), (7828, 1), (4442, 1), (5359, 1), (1789, 1), (8158, 1)],
 [(5359, 1), (7828, 1), (5943, 1), (6255, 1), (1789, 1), (8158, 1)],
 [(5345, 1), (9803, 1), (5359, 2), (9971, 1), (6255, 1), (8158, 1)],
 [(3649, 1), (6275, 1), (3943, 1), (9777, 2), (7828, 1), (5301, 1), (6653, 1)],
 [(3649, 1),
  (1606, 1),
  (359, 1),
  (6599, 1),
  (9777, 1),
  (5267, 1),
  (5301, 1),
  (9080, 1)]]

In [40]:
corpus=documents.collect()

In [41]:
import gensim

In [43]:
#your code here
lda2 = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=10, update_every=1, chunksize=20000, passes=1)

In [44]:
lda2.print_topics()

[u'0.031*year + 0.013*health + 0.012*company + 0.009*price + 0.009*week + 0.009*person + 0.009*drug + 0.008*time + 0.008*government + 0.007*oil',
 u'0.016*child + 0.013*government + 0.011*year + 0.011*measles + 0.010*state + 0.010*family + 0.009*parent + 0.008*man + 0.008*health + 0.007*day',
 u'0.013*home + 0.012*year + 0.011*news + 0.009*event + 0.009*family + 0.007*person + 0.007*official + 0.007*thing + 0.007*college + 0.006*wife',
 u'0.010*country + 0.008*year + 0.008*student + 0.007*law + 0.007*leader + 0.007*person + 0.007*risk + 0.006*health + 0.006*group + 0.006*state',
 u'0.013*drug + 0.009*time + 0.009*health + 0.009*school + 0.008*state + 0.007*official + 0.007*person + 0.007*family + 0.007*business + 0.006*life',
 u'0.025*year + 0.023*percent + 0.013*growth + 0.012*economy + 0.010*health + 0.009*market + 0.008*bank + 0.007*sale + 0.007*spending + 0.007*profit',
 u'0.033*health + 0.016*year + 0.015*care + 0.015*state + 0.014*doctor + 0.013*case + 0.012*law + 0.011*official 